# Other utility functions

This Jupyter notebook contains reusable helper functions for beam deflection analysis.

The implemented functions are used to compute:
- velocity
- acceleration
- bending moment
  
Only applicable for Euler-Bernoulli beam theory

These functions are intended to be **copied or imported** into other notebooks where they are required.

In [ ]:
def get_model_prediction(model, x_array, t_array, x_normalizer, t_normalizer):
    """
    Compute u(x,t) from model for all x and t.

    Returns:
        u_pred: torch.Tensor of shape [Nt, Nx]
    """
    Nx = len(x_array)
    Nt = len(t_array)

    Xg, Tg = np.meshgrid(x_array, t_array, indexing="xy")
    x_flat = Xg.ravel()
    t_flat = Tg.ravel()

    x_tensor = torch.tensor(x_flat, dtype=torch.float32)
    t_tensor = torch.tensor(t_flat, dtype=torch.float32, requires_grad=True)
    bias_tensor = torch.ones_like(x_tensor)

    # Normalize
    x_norm = x_normalizer.normalize(x_tensor)
    t_norm = t_normalizer.normalize(t_tensor)

    inputs = torch.stack([x_norm, t_norm, bias_tensor], dim=1)

    u_pred = model(inputs)[:, 1]  # u channel
    u_pred = u_pred.view(Nt, Nx)  # reshape [Nt, Nx]

    return u_pred, t_tensor, x_tensor

In [ ]:
#Velocity (first derivative of deflection with respect to time )

def compute_velocity(u_pred, t_tensor, Nx):
    """
    Compute du/dt for each x and t
    """
    Nt = u_pred.shape[0]

    velo = torch.autograd.grad(
        outputs=u_pred,
        inputs=t_tensor,
        grad_outputs=torch.ones_like(u_pred),
        create_graph=True,
        retain_graph=True
    )[0]

    velo = velocity.view(Nt, Nx)
    return velo

In [ ]:
#Acceleration (second derivative of deflection with respect to time )
def compute_acceleration(u_pred, t_tensor, Nx):
    """
    Compute d^2u/dt^2 for each x and t
    """
    Nt = u_pred.shape[0]

    # First derivative
    du_dt = torch.autograd.grad(
        outputs=u_pred,
        inputs=t_tensor,
        grad_outputs=torch.ones_like(u_pred),
        create_graph=True,
        retain_graph=True
    )[0]

    # Second derivative
    d2u_dt2 = torch.autograd.grad(
        outputs=du_dt,
        inputs=t_tensor,
        grad_outputs=torch.ones_like(du_dt),
        create_graph=True,
        retain_graph=True
    )[0]

    d2u_dt2 = d2u_dt2.view(Nt, Nx)
    return d2u_dt2

In [ ]:
#Bending moment (second derivative of deflection with respect to space multiplied with EI)
def compute_bending_moment(u_pred, x_tensor, Nx, EI):
    """
    Compute bending moment M = -EI * d^2u/dx^2
    """
    Nt = u_pred.shape[0]

    # First derivative wrt x
    du_dx = torch.autograd.grad(
        outputs=u_pred,
        inputs=x_tensor,
        grad_outputs=torch.ones_like(u_pred),
        create_graph=True,
        retain_graph=True
    )[0]

    # Second derivative wrt x
    d2u_dx2 = torch.autograd.grad(
        outputs=du_dx,
        inputs=x_tensor,
        grad_outputs=torch.ones_like(du_dx),
        create_graph=True,
        retain_graph=True
    )[0]

    d2u_dx2 = d2u_dx2.view(Nt, Nx)
    M = -EI * d2u_dx2
    return M

In [ ]:
#Animation function
def animate_u_over_time(x, t_list, pred_multi, u_normalizer, u_func, colors, interval=5000):
    """
    Create an animation of exact vs model-predicted u(x,t) over time.

    Args:
        x (np.ndarray): 1D array of x positions, shape [Nx].
        t_list (np.ndarray): 1D array of times, shape [Nt].
        pred_multi (np.ndarray): Predicted outputs, shape [Nt, Nx, C], u in channel 1.
        u_normalizer: normalizer with .denormalize(tensor) for u.
        u_func (callable): exact solution u(x,t).
        colors (list): color list (exact, prediction).
        interval (int): milliseconds between frames.

    Returns:
        ani (FuncAnimation): the animation object.
    """


    from matplotlib.animation import FuncAnimation

    Nt, Nx, C = pred_multi.shape
    assert C >= 2, "Expected u in channel 1."

    # --- Denormalize predictions ---
    u_pred_norm = pred_multi[:, :, 1]                         # shape [Nt, Nx]
    u_pred = u_normalizer.denormalize(
        torch.tensor(u_pred_norm, dtype=torch.float32)
    ).numpy()                                                # shape [Nt, Nx]

    # --- Setup figure ---
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.set_xlabel("x")
    ax.set_ylabel("u(x,t)")
    ax.grid(alpha=0.3)
    ax.set_ylim(-1.5, 1.5)
    # Initial lines
    exact_line, = ax.plot(x, u_func(x, t_list[0]), 
                          label="Exact u", color=colors[0])
    pred_line,  = ax.plot(x, u_pred[0], 
                          label="NN u", linestyle="--", color=colors[1])

    ax.legend()
    ax.set_title(f"t = {t_list[0]:.3f}")

    # --- Update function ---
    def update(frame):
        exact_line.set_ydata(u_func(x, t_list[frame]))
        pred_line.set_ydata(u_pred[frame])
        ax.set_title(f"t = {t_list[frame]:.3f}")
        return exact_line, pred_line

    # --- Build animation ---
    ani = FuncAnimation(fig, update, frames=len(t_list), interval=interval)
    plt.close(fig)  # prevents duplicate static figure in notebooks

    return ani

In [ ]:
#ani = animate_u_over_time(
    #x_test, 
    #t_tests, 
    #pred_test_multi, 
    #u_normalizer, 
    #u, 
    #colors,
    #interval=5000   
#)

#from matplotlib.animation import PillowWriter


#writer = PillowWriter(fps=240)


#writer.frame_duration = 1500  

#ani.save("u_animation_trial.gif", writer=writer

